In [0]:
import os

In [0]:
path_train = "train"
path_test = "test"

In [0]:
# get all file names
def getAllFiles(path, type = '.dcm'):
    dcmFiles = []
    for dirpath, subdirs, files in os.walk(path):
        for x in files:
            if x.endswith(type):
                dcmFiles.append(x[:-4])
    return dcmFiles

In [0]:
os.chdir("/content/gdrive/My Drive")

In [0]:
train_image_names = getAllFiles(path_train, ".jpg")
print(len(train_image_names))
train_image_names[:10]

10675


['1.2.276.0.7230010.3.1.4.8323329.10861.1517875225.1530',
 '1.2.276.0.7230010.3.1.4.8323329.14284.1517875250.579837',
 '1.2.276.0.7230010.3.1.4.8323329.11131.1517875231.325914',
 '1.2.276.0.7230010.3.1.4.8323329.32320.1517875160.6416',
 '1.2.276.0.7230010.3.1.4.8323329.3513.1517875178.173087',
 '1.2.276.0.7230010.3.1.4.8323329.5644.1517875189.278405',
 '1.2.276.0.7230010.3.1.4.8323329.384.1517875162.674411',
 '1.2.276.0.7230010.3.1.4.8323329.5184.1517875186.877352',
 '1.2.276.0.7230010.3.1.4.8323329.14432.1517875252.85565',
 '1.2.276.0.7230010.3.1.4.8323329.13169.1517875243.990757']

In [0]:
# we don't have labels for test images
test_image_names = getAllFiles(path_test, ".jpg")
test_image_names[:10]

['1.2.276.0.7230010.3.1.4.8323329.6771.1517875200.76102',
 '1.2.276.0.7230010.3.1.4.8323329.5911.1517875192.469371',
 '1.2.276.0.7230010.3.1.4.8323329.5932.1517875192.889920',
 '1.2.276.0.7230010.3.1.4.8323329.6080.1517875196.313091',
 '1.2.276.0.7230010.3.1.4.8323329.6924.1517875201.945689',
 '1.2.276.0.7230010.3.1.4.8323329.6657.1517875199.426140',
 '1.2.276.0.7230010.3.1.4.8323329.6460.1517875198.378416',
 '1.2.276.0.7230010.3.1.4.8323329.6236.1517875197.214350',
 '1.2.276.0.7230010.3.1.4.8323329.652.1517875163.896438',
 '1.2.276.0.7230010.3.1.4.8323329.6235.1517875197.217889']

In [0]:
import pandas as pd

In [0]:
labels_path = "/content/gdrive/My Drive/train-rle.csv"

In [0]:
labels = pd.read_csv(labels_path)
labels.head()

,ImageId,EncodedPixels
0,1.2.276.0.7230010.3.1.4.8323329.5597.151787518...,-1
1,1.2.276.0.7230010.3.1.4.8323329.12515.15178752...,-1
2,1.2.276.0.7230010.3.1.4.8323329.4904.151787518...,175349 7 1013 12 1009 17 1005 19 1003 20 1002...
3,1.2.276.0.7230010.3.1.4.8323329.32579.15178751...,407576 2 1021 7 1015 10 1013 12 1011 14 1008 ...
4,1.2.276.0.7230010.3.1.4.8323329.32579.15178751...,252069 1 1021 3 1020 4 1018 5 1018 6 1016 7 1...


In [0]:
# convert ImageId dataframe to list
labels_list = labels["ImageId"].values.tolist()

# remove duplicate ImageIds
labels_list = set(labels_list)

In [0]:
print("num labels from csv file: ", len(labels_list))
print("num train images", len(train_image_names))
print("num test images", len(test_image_names))
print(len(train_image_names) + len(test_image_names))

num labels from csv file:  10675
num train images 10675
num test images 1373
12048


In [0]:
# number of image matches
match_count = 0
for image in train_image_names:
  for im in labels_list:
    if image == im:
      match_count += 1

print(match_count)    # not all train images have labels

10675


In [0]:
os.chdir("/content/gdrive/My Drive/train")

In [0]:
def add_extension(aList, extension=".jpg"):
  
  for i in range(len(aList)):
    aList[i] += extension
    aList[i].strip()
  
  return aList 

In [0]:
# delete images that have no labels
images_no_label = list(set(train_image_names) - labels_list)

# add back extension
for i in range(len(images_no_label)):
  images_no_label[i] += ".jpg"
  images_no_label[i].strip()

print(images_no_label)

for im in images_no_label:
  os.remove(im)

['1.2.276.0.7230010.3.1.4.8323329.4843.1517875185.73985.jpg', '1.2.276.0.7230010.3.1.4.8323329.11577.1517875233.694347.jpg', '1.2.276.0.7230010.3.1.4.8323329.4373.1517875182.554858.jpg', '1.2.276.0.7230010.3.1.4.8323329.11566.1517875233.640521.jpg']


In [0]:
os.chdir("/content/gdrive/My Drive")

In [0]:
train_image_names = getAllFiles(path_train, ".jpg")
len(train_image_names)

10675

In [0]:
# labels correspoinding to image order in train folder
train_labels = []
for image in train_image_names:
  for im in labels_list:
    if image == im:
      # dataframe where image name matches
      # basically just one row
      df = labels[labels.ImageId == image]
      if df.iloc[0, 1].strip() == "-1":
        train_labels.append("0")
      else:
        train_labels.append("1")

In [0]:
len(train_labels)

10675

In [0]:
# create pd.dataframe that combines train image names with their labels
df_train = pd.DataFrame(train_image_names)
df_train.columns = ["ImageId"]
df_train["PneumothoraxOrNot"] = pd.DataFrame(train_labels)
df_train.head()

,ImageId,PneumothoraxOrNot
0,1.2.276.0.7230010.3.1.4.8323329.10861.15178752...,0
1,1.2.276.0.7230010.3.1.4.8323329.14284.15178752...,0
2,1.2.276.0.7230010.3.1.4.8323329.11131.15178752...,0
3,1.2.276.0.7230010.3.1.4.8323329.32320.15178751...,0
4,1.2.276.0.7230010.3.1.4.8323329.3513.151787517...,0


In [0]:
pneumothorax_images = df_train[df_train.PneumothoraxOrNot == "1"]
pneumothorax_images = pneumothorax_images.ImageId.to_list()
print("num pneumothorax images:", len(pneumothorax_images))

non_pneumothorax_images = df_train[df_train.PneumothoraxOrNot == "0"]
non_pneumothorax_images = non_pneumothorax_images.ImageId.to_list()
print("num non pneumothorax images images:", len(non_pneumothorax_images))

num pneumothorax images: 2379
num non pneumothorax images images: 8296


In [0]:
# add back extension
add_extension(pneumothorax_images)
add_extension(non_pneumothorax_images)
print(pneumothorax_images[:5])
print(non_pneumothorax_images[:5])

['1.2.276.0.7230010.3.1.4.8323329.13169.1517875243.990757.jpg', '1.2.276.0.7230010.3.1.4.8323329.31650.1517875156.50390.jpg', '1.2.276.0.7230010.3.1.4.8323329.3486.1517875178.51764.jpg', '1.2.276.0.7230010.3.1.4.8323329.12413.1517875239.20295.jpg', '1.2.276.0.7230010.3.1.4.8323329.2166.1517875171.437026.jpg']
['1.2.276.0.7230010.3.1.4.8323329.10861.1517875225.1530.jpg', '1.2.276.0.7230010.3.1.4.8323329.14284.1517875250.579837.jpg', '1.2.276.0.7230010.3.1.4.8323329.11131.1517875231.325914.jpg', '1.2.276.0.7230010.3.1.4.8323329.32320.1517875160.6416.jpg', '1.2.276.0.7230010.3.1.4.8323329.3513.1517875178.173087.jpg']


In [0]:
os.chdir("/content/gdrive/My Drive/train")

In [0]:
!mkdir pneumothorax
!mkdir nonpneumothorax

In [0]:
import shutil

In [0]:
def move_files(files, dest_folder):
  for f in files:
    shutil.move(f, dest_folder)

In [0]:
# separate pneumothorax and nonpneumothorax images to subfolders
pneumothorax_dest = "/content/gdrive/My Drive/train/pneumothorax"
nonpneumothorax_dest = "/content/gdrive/My Drive/train/nonpneumothorax"

In [0]:
move_files(pneumothorax_images, pneumothorax_dest)

In [0]:
move_files(non_pneumothorax_images, nonpneumothorax_dest)

In [0]:
yes = getAllFiles("/content/gdrive/My Drive/train/pneumothorax", ".jpg")
print(len(yes) == len(pneumothorax_images))
no = getAllFiles("/content/gdrive/My Drive/train/nonpneumothorax", ".jpg")
print(len(no) == len(non_pneumothorax_images))

True
True
